<a href="https://colab.research.google.com/github/krishnasd1007/sea_ice/blob/main/random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# **DataEngineering**

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

path = '/gdrive/My Drive/sea_ice/'
data = pd.read_csv(path + 'DRIFT_DATA_TRAIN.csv')

data['Ui'] = np.sqrt((data['u_buoy']*data['u_buoy'])+(data['v_buoy']*data['v_buoy']))
#data['Uw'] = np.sqrt((data['x_EASE']*data['x_EASE'])+(data['y_EASE']*data['y_EASE']))
data['Ua'] = np.sqrt((data['u_ERA5']*data['u_ERA5'])+(data['v_ERA5']*data['v_ERA5']))
#data['Uiwx'] = data['u_buoy'] - data['x_EASE']
#data['Uiwy'] = data['v_buoy'] - data['y_EASE']
#data['Uiw'] = np.sqrt((data['Uiwx']*data['Uiwx'])+(data['Uiwy']*data['Uiwy']))
#data['dot'] = (data['u_buoy']*data['Uiwx']) + (data['v_buoy']+data['Uiwy'])
#data['denom'] = data['Ua'] * data['Uiw']
#data['th'] = data['dot'] / data['denom']
#data['th'] = (data['th'] - data['th'].min()) / (data['th'].max() - data['th'].min()) 
#data['theta'] = (np.arccos(data['th'])) * (180/np.pi)
data = data.drop(columns=['year','month','day','doy','h_cs2smos','id_buoy','u_buoy','v_buoy'])
data.head(2)

,x_EASE,y_EASE,u_ERA5,v_ERA5,sic_CDR,h_piomas,d2c,Ui,Ua
0,147.506958,138.582672,-6.704156,-0.321260,0.990195,3.189743,522.523298,1.370671,6.711849
1,146.834778,120.509880,-6.818630,-0.674205,0.966372,2.484009,412.767669,0.741408,6.851881


In [24]:
data.isnull().sum(axis = 0)

x_EASE      0
y_EASE      0
u_ERA5      0
v_ERA5      0
sic_CDR     0
h_piomas    0
d2c         0
Ui          0
Ua          0
dtype: int64

# **Model for speed**

In [25]:
x = data.drop(columns = ['Ui'])
y = data['Ui']

In [26]:
from sklearn.ensemble import RandomForestRegressor 

model_speed = RandomForestRegressor(n_estimators = 100, random_state = 0) 
model_speed.fit(x, y) 

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [27]:
from sklearn.metrics import mean_squared_error 

pred_y = model_speed.predict(x)
mse =mean_squared_error(y, pred_y)

In [28]:
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 1.5298384146346158


#**Test set prediction**

In [29]:
datatest = pd.read_csv(path + 'DRIFT_DATA_TEST_hackathon.csv')

datatest['Uw'] = np.sqrt((datatest['x_EASE']*datatest['x_EASE'])+(datatest['y_EASE']*datatest['y_EASE']))
datatest['Ua'] = np.sqrt((datatest['u_ERA5']*datatest['u_ERA5'])+(datatest['v_ERA5']*datatest['v_ERA5']))
datatest['Ui'] = np.sqrt((datatest['u_buoy']*datatest['u_buoy'])+(datatest['v_buoy']*datatest['v_buoy']))

datatest = datatest.drop(columns=['year','month','day','doy','h_cs2smos','id_buoy','u_buoy','v_buoy'])

In [30]:
x = datatest.drop(columns=['Ui'])
y = datatest['Ui']

In [31]:
from sklearn.ensemble import RandomForestRegressor 

model_speed = RandomForestRegressor(n_estimators = 100, random_state = 0) 
model_speed.fit(x, y) 

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [32]:
from sklearn.metrics import mean_squared_error 

pred_y = model_speed.predict(x)
mse =mean_squared_error(y, pred_y)

In [33]:
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 1.6825064063041657


In [35]:
prediction_vel = model_speed.predict(x)
np.savetxt(path + "prediction_velocityrf.csv",prediction_vel, delimiter=",")